In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [3]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 1064, done.
remote: Counting objects: 100% (323/323), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1064 (delta 210), reused 152 (delta 146), pack-reused 741
Receiving objects: 100% (1064/1064), 2.28 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (617/617), done.
/content/easy-few-shot-learning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/easy-few-shot-learning
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for easyfsl: filename=easyfsl-1.3.0-py3-none-any.whl size=52364 sha256=cf2a4a54b42e36b1f0756fd2f41bbb9fa843d408cd655a74fa935f73d918ceca
  Stored in directory: /root/.cache/pip/wheels/bb/79/35/1da2d9ffa9c9b6892f0cca4cda77ee397b97a1deda949049e2
Successfully built easyfsl


In [4]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import torchvision
import torch.utils.data

In [5]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Episodic Training

In [6]:
n_way = 5
n_shot = 5
n_query = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_workers = 0

In [7]:
image_size = 28

# Instantiate the datasets
train_set = torchvision.datasets.GTSRB(
    root='/content/data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True)

100%|██████████| 187490228/187490228 [00:08<00:00, 22265206.99it/s]


Extracting /content/data/gtsrb/GTSRB-Training_fixed.zip to /content/data/gtsrb


In [8]:
!mkdir /content/data/gtsrb/GTSRB/Testing

!mv /content/data/gtsrb/GTSRB/Training/00021 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00037 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00042 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00016 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00006 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00005 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00004 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00035 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00000 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00029 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00010 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00012 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00040 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00009 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00036 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00031 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00041 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00018 /content/data/gtsrb/GTSRB/Testing

In [9]:
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 500
n_validation_tasks = 100
image_size = 28

# Instantiate the datasets
train_set = torchvision.datasets.GTSRB(
    root='/content/data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=False)

train_set_size = int(len(train_set) * 0.8)
val_set_size = len(train_set) - train_set_size
train_set, val_set = torch.utils.data.random_split(train_set, [train_set_size, val_set_size])

train_set.get_labels = lambda: [
    instance[1] for instance in train_set
]

val_set.get_labels = lambda: [
    instance[1] for instance in val_set
]

# Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

# Finally, the DataLoader. We customize the collate_fn so that batches are delivered
# in the shape: (support_images, support_labels, query_images, query_labels, class_ids)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

print(train_set_size,val_set_size)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


12768 3192


In [ ]:
#Only used this encoder for RelationNetworks few shot algorithm

class CNNEncoder(nn.Module):
    """docstring for ClassName"""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())
        self.layer4 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        #out = out.view(out.size(0),-1)
        return out # 64

In [10]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier
from easyfsl.modules import resnet12


convolutional_network = resnet12()
few_shot_classifier = PrototypicalNetworks(convolutional_network).to(DEVICE)

In [11]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [12]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))

            loss.backward()

            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [13]:
from easyfsl.utils import evaluate


best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = few_shot_classifier.state_dict()
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

    if epoch % 10 == 0 : torch.save(best_state, '/content/drive/MyDrive/pr_final_presentation/weights/PrototypicalNetworks_5_5.pth')

Epoch 0


Training:   0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Validation: 100%|██████████| 100/100 [00:05<00:00, 18.57it/s, accuracy=0.98]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|██████████| 100/100 [00:05<00:00, 18.26it/s, accuracy=0.987]


Ding ding ding! We found a new best model!
Epoch 2


...

Validation: 100%|██████████| 100/100 [00:06<00:00, 16.23it/s, accuracy=0.996]


Epoch 198


Validation: 100%|██████████| 100/100 [00:05<00:00, 18.13it/s, accuracy=0.996]


Epoch 199


Validation: 100%|██████████| 100/100 [00:05<00:00, 18.01it/s, accuracy=0.998]


In [14]:
PATH = '/content/drive/MyDrive/pr_final_presentation/weights/PrototypicalNetworks_5_5.pth'
few_shot_classifier.load_state_dict(torch.load(PATH))

<All keys matched successfully>

**Evaluation**

In [15]:
import os

old_folder_name = '/content/data/gtsrb/GTSRB/Training'
new_folder_name = '/content/data/gtsrb/GTSRB/Trainingg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

old_folder_name = '/content/data/gtsrb/GTSRB/Testing'
new_folder_name = '/content/data/gtsrb/GTSRB/Training'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

In [16]:
n_test_tasks = 1000

test_set = torchvision.datasets.GTSRB(
    root='/content/data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=False
    )

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [17]:
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 1000/1000 [00:58<00:00, 17.22it/s, accuracy=0.881]

Average accuracy : 88.08 %


# Classical Training

In [ ]:
image_size = 28

# Instantiate the datasets
train_set = torchvision.datasets.GTSRB(
    root='./data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True)
train_set_size = int(len(train_set) * 0.8)
val_set_size = len(train_set) - train_set_size
train_set, val_set = torch.utils.data.random_split(train_set, [train_set_size, val_set_size])

train_set.get_labels = lambda: [
    instance[1] for instance in train_set
]

val_set.get_labels = lambda: [
    instance[1] for instance in val_set
]

In [ ]:
# from easyfsl.datasets import CUB
from torch.utils.data import DataLoader

batch_size = 128
n_workers = 0

# train_set = CUB(split="train", training=True)
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=n_workers,
    pin_memory=True,
    shuffle=True,
)

In [ ]:
from easyfsl.modules import resnet12

DEVICE = "cuda"

model = resnet12(
    use_fc=True,
    num_classes=len(set(train_set.get_labels())),
).to(DEVICE)

In [ ]:
from easyfsl.methods import PrototypicalNetworks,Finetune
from easyfsl.samplers import TaskSampler

n_way = 5
n_shot = 5
n_query = 10
n_validation_tasks = 500

# val_set = CUB(split="val", training=False)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

few_shot_classifier = Finetune(model).to(DEVICE)

In [ ]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [150, 180]
scheduler_gamma = 0.1
learning_rate = 1e-01
tb_logs_dir = Path(".")

train_optimizer = SGD(
    model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [ ]:
def training_epoch(model_: nn.Module, data_loader: DataLoader, optimizer: Optimizer):
    all_loss = []
    model_.train()
    with tqdm(data_loader, total=len(data_loader), desc="Training") as tqdm_train:
        for images, labels in tqdm_train:
            optimizer.zero_grad()

            loss = LOSS_FUNCTION(model_(images.to(DEVICE)), labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [ ]:
from easyfsl.methods.tim import FewShotClassifier
from easyfsl.utils import evaluate


best_state = model.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    few_shot_classifier.backbone.requires_grad_(True)
    average_loss = training_epoch(model, train_loader, train_optimizer)

    if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
        model.set_use_fc(False)
        few_shot_classifier.backbone.requires_grad_(False)
        validation_accuracy = evaluate(
            few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
        )
        model.set_use_fc(True)
        few_shot_classifier.backbone.requires_grad_(True)

        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_state = model.state_dict()
            print("Ding ding ding! We found a new best model!")

        tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)
        # print(validation_accuracy)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()
    if epoch % 10 == 0 : torch.save(best_state, '/content/drive/MyDrive/pr_final_presentation/weights/Finetune_5_5.pth')

Epoch 0


Training: 100%|██████████| 167/167 [00:26<00:00,  6.42it/s, loss=1.14]


Epoch 1


Training: 100%|██████████| 167/167 [00:25<00:00,  6.47it/s, loss=0.0646]


Epoch 2


...

Training: 100%|██████████| 167/167 [00:25<00:00,  6.53it/s, loss=0.0125]


Epoch 108


Training: 100%|██████████| 167/167 [00:25<00:00,  6.52it/s, loss=0.00588]


Epoch 109


Validation:   4%|▍         | 20/500 [00:01<00:36, 13.21it/s, accuracy=1]

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/pr_final_presentation/weights/Finetune_5_5.pth'))

In [ ]:
n_test_tasks = 1000

# test_set = CUB(split="test", training=False)

test_set = torchvision.datasets.GTSRB(
    root='./data',
    split='test',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True
    )

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [ ]:
model.set_use_fc(False)
few_shot_classifier.backbone.requires_grad_(False)
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")